# **Retrieval Augmented Generation**

_Taalmodellen zijn heel krachtig, maar zijn qua kennis gelimiteerd tot hun trainingdata en kunnen halucineren. De antwoorden zijn niet gestaafd door een bronvermelding. Vragen over onze eigen documenten kan deze ook niet zomaar beantwoorden. Dit is waar Retrieval Augmented Generation (RAG) een oplossing biedt. Het is een techniek waarbij we onze eigen documenten kunnen bevragen met natuurlijke taal._

<img src="../.github/rag.png" alt="RAG" width="600"/>

---

## **Voorbereiding**

In [41]:
%pip install pymilvus[model] pypdf ollama --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---

## **Embedder**

In [50]:
from pymilvus import model

# Laadt een standaard embedder model
embedder = model.DefaultEmbeddingFunction()

print(f"Elke vector zal uit {embedder.dim} kommagetallen bestaan.")

Elke vector zal uit 768 kommagetallen bestaan.


---

## **Vector database**

Zoals verteld tijdens de theorie, hebben we een **vector database** nodig om onze documenten in op te slaan.


Dit kan op verschillende manieren:

| Opslagmethode     | Gebruik                   |
|-------------------|---------------------------|
| ☁️ Cloud             | Productie                 |
| 🐋 Docker container  | Productie (eigen server)  |
| 📁 Lokaal bestand    | Development               |
| 🕓 In-memory         | Development               |

Wij gaan **Milvus** gebruiken, een open-source vector database.<br>
Net zoals Ollama, kan je Milvus zowel via een CLI als via Python aanspreken.<br>
Naast Milvus zijn er tal van andere opties beschikbaar zoals Chroma, Qdrant, Vespa, (Pinecone, Weaviate) ...


### **1. Verbinden met / aanmaken van vector database**

Typisch zou dit er ongeveer zo uitzien:

```py
vectordb = MilvusClient(uri="http://localhost:19530", username="admin", password="password")
```

Wij hebben geen online database die ergens draait en toegankelijk is via een URL. <br>
In plaats daarvan gaan we een **lokaal bestand** aanmaken dat we kunnen gebruiken als database.<br>
Als het bestand niet bestaat, dan wordt het automatisch aangemaakt, handig!

In [43]:
from pymilvus import MilvusClient

vectordb = MilvusClient("../milvus.db")

Met het client object `vectordb` kunnen we nu d.m.v. Python code interageren met de Milvus database:
- `vectordb.create_user(...)`
- `vectordb.create_collection(...)`
- `vectordb.insert(...)`
- `vectordb.query(...)`
- ...

### **2. Maak een collectie aan**

In [44]:
collections = vectordb.list_collections()

if "blogpost" not in collections:
  # Aanmaken van nieuwe collectie
  vectordb.create_collection(
    collection_name="blogpost",
    dimension=embedder.dim
  )

# Lijst alle collecties op ter controle
collections = vectordb.list_collections()
print(f"Beschikbare collecties: {', '.join(collections)}")

Beschikbare collecties: blogpost


### **3. Uitlezen van PDF**

<img src="../.github/tekst-extractie.png" alt="Tekst extractie" width="400"/>

Nu we onze vector database hebben aangemaakt, kunnen we beginnen met het toevoegen van documenten. <br>
We gaan een PDF inlezen, omzetten naar vectoren en deze toevoegen aan de database.

In [51]:
from pypdf import PdfReader

def lees_pdf(path):
    # Open de PDF
    pdf = PdfReader(path)
    text = ""
    # Overloop elke pagina
    for page in pdf.pages:
        # Lees de paginatekst uit
        page_text = page.extract_text()
        # Voeg de paginatekst toe aan de totale tekst
        text += page_text + "\n"
    return text

In [52]:
text = lees_pdf("Blogpost.pdf")

print(text)

Blogpost
Hoe maak je AI-modellen klein en 
efficiënt?
AI-modellen worden steeds krachtiger, maar vaak ook steeds groter. Grote 
modellen vereisen veel rekenkracht, geheugen en energie - wat ze minder 
geschikt maakt voor gebruik op edge devices, maar ook realtime computervisie 
applicaties zoals de Howest Virtual Mirror. Gelukkig zijn er diverse technieken 
om AI-modellen te verkleinen zonder de accuraatheid ervan te verlagen. In 
deze blogpost bespreken we de belangrijkste optimalisatiestrategieën: 
quantisatie, distillatie, pruning en meer. We bekijken ook welke tools je kunt 
gebruiken, op welke platformen ze draaien, en hoe groot de impact kan zijn.
1. Quantisatie
Inleiding
Binnenin een AI model zitten er miljoenen tot miljarden kommagetallen - ook 
wel “gewichtenˮ en “activatiesˮ genoemd. Elk getal neemt typisch 32 bits 
(nullen en enen) geheugen in beslag.
Voor een computer zijn kommagetallen eigenlijk zeer lastig om mee te rekenen. 
Één zoʼn berekening duurt uiteraard slechts en

### **4. Tekst opsplitsen in chunks**

In [ ]:
def verdeel_in_chunks(text):
    # Kap de tekst in stukken van 512 karakters met een overlap van 128 karakters
    return [text[i:i+512] for i in range(0, len(text), 512-128)]

chunks = verdeel_in_chunks(text)

print(f"De tekst van {len(text)} karakters is opgedeeld in {len(chunks)} stukken van 512 karakters met een overlap van 128 karakters.")

### **5. Chunks omzetten naar vectoren (embedding)**

<img src="../.github/tekst-embedding.png" alt="Tekst embedding" width="400"/>

In [ ]:
# Omzetten van teksten naar vectoren
vectors = embedder.encode_documents(chunks)

# Een kijkje nemen naar de eerste vector = chunk 1
print(vectors[0])

[ 1.29018397e-02  3.39037056e-02  1.38407705e-02 -2.37033750e-02
  1.41432738e-02  3.95084573e-03  1.88760994e-02 -8.68979462e-03
  1.15502403e-02 -5.03735965e-02  2.82360528e-02  1.72524620e-02
 -4.01088364e-02 -9.24565788e-02  4.08351003e-03  8.70586708e-03
 -4.49316399e-02 -2.97982155e-02  9.14868732e-03 -3.51780520e-02
 -2.84741317e-02  7.10664068e-03  6.14569991e-03 -4.83656797e-02
 -1.91956371e-02  2.58579654e-02 -2.50019176e-03  6.60994206e-03
 -2.69456916e-02 -6.56962520e-03 -6.13001482e-02 -1.31405158e-02
 -1.11234524e-03 -1.92446541e-02 -5.01197160e-02 -5.89044375e-02
 -4.28711448e-03  1.35903676e-02 -1.09149342e-02  4.75778749e-02
 -1.04830449e-02  1.61411409e-02  5.04271557e-03  9.73946011e-02
 -1.69800678e-02  4.72020984e-02  3.78694586e-02 -1.64446858e-02
 -3.84101321e-02 -4.90670064e-02  5.57477220e-02 -2.05239546e-02
 -1.50153661e-02  2.02639093e-02 -3.07706504e-02  1.58992888e-02
  2.00894465e-02 -5.43083171e-02  1.18508442e-02 -2.68718704e-02
  1.21450267e-02 -2.39214

### **6. Vectoren in database stoppen**

<img src="../.github/vectors-in-database.png" alt="Tekst embedding" width="500"/>

In [9]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

# Formatteer de vectoren als een lijst van dictionaries
data = [ {"text": text, "vector": vector, "id": id} for id, (text, vector) in enumerate(zip(chunks, vectors)) ]

# Vectoren toevoegen aan de blogpost collectie
client.insert("blogpost", data)

{'insert_count': 13, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'cost': 0}

---

## **Vector database bevragen** *(= Retrieval)*

### **6. Vraag embedden**

<img src="../.github/vraag-embedden.png" alt="RAG" width="400"/>

In [37]:
question = "Wat kan je me vertellen over de blogpost?"

question_vector = embedder.encode_queries([question])[0]

### **7. Relevante documenten zoeken**

<img src="../.github/relevante-docs-zoeken.png" alt="RAG" width="400"/>

In [38]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

results = client.search(
    collection_name="blogpost",
    data=[question_vector],
    output_fields=["text"]
)

print("Resultaten:")
for result in results[0]:
    print(result)

Resultaten:
{'id': 0, 'distance': 0.43110722303390503, 'entity': {'text': 'Blogpost\nHoe maak je AI-modellen klein en \nefficiënt?\nAI-modellen worden steeds krachtiger, maar vaak ook steeds groter. Grote \nmodellen vereisen veel rekenkracht, geheugen en energie - wat ze minder \ngeschikt maakt voor gebruik op edge devices, maar ook realtime computervisie \napplicaties zoals de Howest Virtual Mirror. Gelukkig zijn er diverse technieken \nom AI-modellen te verkleinen zonder de accuraatheid ervan te verlagen. In \ndeze blogpost bespreken we de belangrijkste optimalisatiestrategieën: \nquanti'}}
{'id': 12, 'distance': 0.3301771879196167, 'entity': {'text': 'putervisie en taalmodellen maken ze het \nverschil.\nHet vergt wel enige tijd om mee te experimenteren en het verlies in \naccuraatheid te beoordelen en/of te compenseren. Deze extra ontwikkelingstijd \nkan afhankelijk van de schaal van het project al dan niet de moeite waard zijn.\nBlogpost\n5\n'}}
{'id': 1, 'distance': 0.310672581195

---

## **Antwoord formuleren** *(= Generation)*

### **8. Taalmodel bevragen**

In [ ]:
from ollama import chat

prompt_template = """
Je bent een professionele assistent. Gebruik onderstaande context om de vraag te beantwoorden.
Als het antwoord niet in de context staat, zeg dan dat je het niet weet.

### Context:
{context}

### Vraag:
{question}

### Antwoord:
"""

def vraag_ollama_rag(context, question, model="llama3"):
    response = chat(model=model, messages=[{"role": "user", "content": prompt_template.format(context=context, question=question)}])
    return response.message.content.strip()

# Chunks van gevonden documenten terug aan elkaar plakken om context te vormen
context = "\n\n".join([result.entity.text.strip() for result in results[0]])

# Vraag stellen aan Ollama met de context en de vraag
answer = vraag_ollama_rag(context, question)

print(answer)

Deze blogpost behandelt verschillende technieken om AI-modellen klein en efficiënt te maken zonder hun accuratesse te verliezen. De belangrijkste optimalisatiestrategieën die worden besproken zijn quantisatie, pruning en distillatie. Quantisatie verwijdert niet-binaire getallen uit het model, pruning verwijdert onnodige elementen uit het model en distillatie traineert een klein model om de output van een groot model te imiteren.

De blogpost geeft ook voorbeelden van hoe deze technieken worden toegepast in praktijk, zoals TinyBERT, die 7 keer kleiner en 9 keer sneller is dan het oorspronkelijke BERT-model terwijl het slechts een kleine daling van ±3% heeft in nauwkeurigheid.

Verder wordt er ook gesproken over frameworks en libraries die kunnen worden gebruikt om modellen te optimaliseren, zoals PyTorch, TensorFlow en Hugging Face's Optimum library.
